In [7]:
ROOT_DIR = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain'
multichainLoc = ''
chainName = 'public_access_3'
datadir = f'{ROOT_DIR}/multichain'
querydir = f'{ROOT_DIR}/public/code'
metafile = f'{ROOT_DIR}/public/data/samples/metadata.csv'
annotation_path = f'{ROOT_DIR}/public/data/annotations'
personPath = f'{ROOT_DIR}/public/data/clinical/person.csv'
dataPath = f'{ROOT_DIR}/public/data/clinical/'

In [6]:
# Standard libaries
import pandas as pd
import json
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
sys.path.append(f'{querydir}')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#Network functions
from QueryClinical import ( 
                            queryGroupDemographics,
                            domainQuery,
                            personQuery,
                            parseKeys
                          )

from QueryVariant import ( 
                            queryVariants,
                            queryPersonsChroms,
                            getPatientVariantAnnotation
                         )

from QueryCombination import (
                                extractGeneVariants,
                                queryClinicalGeneVariantRange,
                                queryVariantClinical,
                             )

from QueryAnalysis import ( 
                            queryMetadata, 
                            queryKinship,
                            querySamplePCA
                          )


## BUILD COHORT

### Harmonize genetic data

In [8]:
def harmonizeMetadata(metadata):
    """ Returns list of harmonized IDs """
    response = queryMetadata(chainName, datadir, metadata)
    metadata_list = metadata.split(',')
    ids = [value for meta in metadata_list for value in list(response[meta].values())[0]]
    meta_ids = list(set(ids))
    print(f'{len(meta_ids)} patients meet sequencing metadata criteria')
    return meta_ids

In [9]:
#Search for patients variant called with GATK and sequenced with Illumina seq machine
metadata = 'GATK,Illumina'
meta_ids = harmonizeMetadata(metadata)

654 patients meet sequencing metadata criteria


### Remove related people

In [11]:
def removeRelated(ids):
    """ Checks and removes related patients """
    if isinstance(ids, list):
        ids = ','.join(ids)
    response = queryKinship(chainName, datadir, ids)
    response_json = json.loads(response)
    kin_df = pd.DataFrame(response_json)
    unrelated = kin_df.apply(lambda col: (col == 'UR').sum() == kin_df.shape[0] - 1)
    unrelated_ids = unrelated.index.tolist()
    print(f'{len(unrelated_ids)} remaining after removing related samples')
    return unrelated_ids

In [12]:
''' THIS STEP TAKES ~1-2MINUTES BECAUSE OF THE CALCUALTION NOT DATA EXTRACTION. 
    IMPLEMENTATION OF GFAF HAS BEEN VECTORIZED BUT CAN FURTHER OPTIMIZED'''
unrelated_ids = removeRelated(meta_ids)

654 remaining after removing related samples


# EXTRACT PC'S

In [13]:
def getPCs(ids, kSearch):
    """ Get PCs for list of samples """
    kSearch = 20
    if isinstance(ids, list):
        ids = ','.join(ids)
    response = querySamplePCA(chainName, datadir, ids, kSearch)
    pc_df = pd.DataFrame(json.loads(response))
    return pc_df

In [14]:
kSearch = 20
pc_df = getPCs(unrelated_ids, kSearch)

{"V1":{"34272":-108.1433257639,"60656":-106.0768393394,"604":-109.0365535663,"51122":-107.0728843615,"68393":-108.6251817978,"37115":-109.1816715588,"55454":-106.5960878482,"64066":-108.1176930274,"68279":-107.1479907927,"57067":-107.7020722075,"38913":-108.5639341595,"81768":-108.8656813728,"64009":-107.1129467284,"72577":-108.6603573982,"78089":-106.8765809933,"94551":-106.8322497175,"62814":-107.9113775425,"65605":-107.975779646,"50434":-107.4149684109,"18692":-105.6965619852,"68672":-106.1904643373,"48761":-107.4758163025,"7489":-106.5238295814,"4316":-107.8200491645,"41481":-108.7445910702,"111072":-109.5962044884,"109029":-105.853345001,"30255":-108.4844648495,"59146":-107.0313600121,"50060":-107.9796268026,"74356":-107.6155012004,"10739":-110.2768100919,"2530":-106.8182933615,"7870":-105.9507578055,"102825":-113.3682937747,"69585":-110.3729841245,"58795":-110.7220423874,"93451":-112.0016976904,"59067":-111.0296842729,"55480":-103.77909191,"37803":-107.9465660987,"713":-110.67575

# Get phenotypes

###  Age and gender

In [15]:
def getAgeGender(chainName, multichainLoc, datadir, ids):
    """ extract the age and gender of all patients """
    searchKeys = 'birth_datetime,gender_concept_id'
    demo_data = queryGroupDemographics(chainName, multichainLoc, datadir, searchKeys)
    demographics = pd.DataFrame(demo_data).T
    keys = searchKeys.split(',')
    demographics.columns = keys

    demographics['birth_datetime'] = pd.to_datetime(demographics['birth_datetime'])
    current_date = pd.to_datetime('2023-07-06')
    demographics['age'] = (current_date - demographics['birth_datetime']).dt.days / 365.25
    demo_processed = demographics[['gender_concept_id', 'age']]
    demo_processed['gender'] = demo_processed['gender_concept_id'].replace({8507:0, 8532:1})
    demo_processed.drop(columns = 'gender_concept_id', inplace = True)

    if isinstance(ids,str):
        ids = ids.split(',')

    return demo_processed.loc[ids]

In [17]:
demos = getAgeGender(chainName, multichainLoc, datadir, unrelated_ids)

### Phenotype of interest

In [18]:
def getPhenotype(pheno_id, demos):
    """ Get phenotype of interest """
    searchKeys = 'demographics' # returns basic information for patients with disease. can be changed if more complex info needed
    response = domainQuery(chainName, multichainLoc, datadir, pheno_id, searchKeys)
    data = [r['data']['json'] for r in response]
    df = pd.DataFrame(data)
    disease_ids = list(df['person_id'].unique())
    phenos = demos.copy()
    phenos['phenotype'] = 0
    phenos.loc[phenos.index.isin(disease_ids), 'phenotype']  = 1
    return phenos

In [19]:
#Phenotype here is diabetes but can be any user defined logic
pheno_id = '201826'
phenos = getPhenotype(pheno_id, demos)

# EXTRACT GENOTYPE INFORMATION

In [30]:
def getVariantDF(chrom, variants, genotype = 'all', metadata = None):
    """ Get variant in DF format """
    response = queryVariants(chainName, multichainLoc, datadir, chrom, variants, genotype, metadata)
    variants_dict = json.loads(response)
    data_for_df = []
    for variant, genotypes in variants_dict.items():
        for genotype, ids in genotypes.items():
            for id_ in ids:
                data_for_df.append({'variant': variant, 'genotype': genotype, 'id': id_})
    df = pd.DataFrame(data_for_df)
    variants_df = df.pivot(index='id', columns='variant', values='genotype').reset_index()
    variants_df.set_index('id', inplace=True)
    variants_df.replace({"0|0":0, "1|0":1, "1|1":1}, inplace = True)
    variants_df.columns = ['variant']
    variants_df.index = variants_df.index.astype(str)
    return variants_df

In [31]:
""" Extract variants of interest
    We show an example variant here to limit computation cost. In a full analysis we would pull the stored variants
    from the mapping stream and loop through them with parallelization  """
chrom = '1'
variant = '111489509'
genotype = 'all'
metadata = None
variants_df = getVariantDF(chrom, variant, genotype = 'all', metadata = None)

{"111489509":{"1|0":[34272,64066,5202,29870,50636,109029,10739,19826,37803,713,110758,42610,50352,56945,91436,109205,6051,116583,44923,77581,18987,113180,55070,76330,38128,58604,37843,86300,40013,76908,96202,69464,34463,8992,16027,76437,32694,19179,59828,69512,15729,15285,97495,41545,25540,80397,83244,73249,67602,2402,52607,68754,91646,62713,97434,48809,65855,103493,5248,64049,62883,15882,4336,8679,75270,27561,72078,72201,82066,54818,80870,49785,12384,69212,55816,13376,22032,13234,62565,49959,103779,9073,85879,53006,10278,95147,26835,114683,17223,38719,2448,78799,69818,65910,88215,4622,85580,55025,87092,81850,49821,108028,34182,75823,29039,100619,29573,31400,89959,40993,81478,18282,67766,111149,66359,89595,100985,101831,39213,46447,59771,67237,20802,19919,110415,3631,44156,65287,11372,40319,44818,64222,17586,26414,39544,81655,3355,99967,24619,106094,53711,85082,65916,100481,24193,116868,107172,12688,10442,48948,114193,48775,9354,29554,35863,56919,20146,90215,107889,77200,33569,96473,82

# RUN GWAS

In [32]:
def runGwas(pc_df, phenos, variants_df):
    #Linear mixed model with age, gender and phenotype
    covariates = pc_df.merge(phenos, left_index =True, right_index=True)
    data = covariates.merge(variants_df, left_index =True, right_index=True)
    formula = f"variant ~ phenotype + age + gender + " + ' + '.join([f'V{i}' for i in range(1,kSearch+1)])
    md = smf.ols(formula, data)
    mdf = md.fit()
    return mdf

In [33]:
mdf = runGwas(pc_df, phenos, variants_df)
mdf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                variant   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     2.577
Date:                Wed, 01 Nov 2023   Prob (F-statistic):           8.47e-05
Time:                        19:51:12   Log-Likelihood:                -331.89
No. Observations:                 654   AIC:                             711.8
Df Residuals:                     630   BIC:                             819.4
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3277      0.074      4.423      0.000       0.182       0.473
phenotype     -0.0084      0.075     -0.111      0.912      -0.156       0.140
age           -0.0002      0.001     -0.265      0.791      -0.001       0.001
gender        -0.0498      0.033     -1.516      0.130      -0.114       0.015
V1             0.0007      0.000      1.555      0.120      -0.000       0.002
V2         -9.001e-05      0.000     -0.435      0.664      -0.000       0.000
V3             0.0013      0.001      0.897      0.370      -0.001       0.004
V4         -3.068e-05      0.001     -0.047      0.963      -0.001       0.001
V5             0.0002      0.001      0.230      0.818      -0.001       0.002
V6            -0.0024      0.005     -0.527      0.599      -0.011       0.006
V7            -0.0010      0.001     -0.870      0.385      -0.003       0.001
V8            -0.0030      0.005     -0.611      0.542      -0.012       0.007
V9            -0.0001      0.001     -0.148      0.883      -0.002       0.002
V10            0.0062      0.003      2.316      0.021       0.001       0.012
V11         4.637e-05      0.001      0.091      0.927      -0.001       0.001
V12           -0.0031      0.002     -1.362      0.174      -0.008       0.001
V13            0.0009      0.002      0.531      0.596      -0.002       0.004
V14           -0.0008      0.001     -0.565      0.573      -0.003       0.002
V15            0.0021      0.002      1.318      0.188      -0.001       0.005
V16           -0.0003      0.002     -0.157      0.875      -0.003       0.003
V17           -0.0010      0.001     -1.008      0.314      -0.003       0.001
V18           -0.0022      0.002     -1.126      0.261      -0.006       0.002
V19         2.743e-05      0.002      0.015      0.988      -0.004       0.004
V20           -0.0022      0.001     -1.652      0.099      -0.005       0.000
==============================================================================
Omnibus:                      101.756   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.356
Skew:                           1.176   Prob(JB):                     1.36e-33
Kurtosis:                       2.837   Cond. No.                         668.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""